# **WQU Capstone Project**

### **Dynamic MACD Trading with Multi-modal Agent**

#### **Student Group 6386**
##### Sunil Sharma ([sunilksh+mscfe@outlook.com](mailto:sunilksh+mscfe@outlook.com))

---

In [1]:
import autogen
from autogen import AssistantAgent, UserProxyAgent
from autogen.agentchat.contrib.multimodal_conversable_agent import MultimodalConversableAgent
from autogen.cache import Cache
from textwrap import dedent

In [2]:
company = "AAKASH"
start_date = "2022-01-01"
end_date = "2024-01-01"
file_path = f'../../data/DailyData/{company}.csv'

In [3]:
config_list_4v = autogen.config_list_from_json(
    "./CONFIG.json",
    filter_dict={
        "model": ["gpt-4o"],
    },
)
config_list = autogen.config_list_from_json(
    "./CONFIG.json",
    filter_dict={
        "model": ["gpt-4o"],
    },
)
llm_config_4v = {"config_list": config_list_4v,"max_tokens":300,"temperature": 0.0}
llm_config = {"config_list": config_list,"max_tokens":300,"temperature": 0.0}

In [4]:
import pandas as pd

class MACDDataProcessor:
    def __init__(self, csv_file_path: str):
        self.data = pd.read_csv(csv_file_path, parse_dates=['date'])
        self.data.set_index('date', inplace=True)

    def filter_data(self, start_date: str, end_date: str):
        self.filtered_data = self.data.loc[start_date:end_date]

    def calculate_ema(self, period: int):
        return self.filtered_data['close'].ewm(span=period, adjust=False).mean()

    def calculate_macd(self):
        ema_fast = self.calculate_ema(12)
        ema_slow = self.calculate_ema(26)
        macd = ema_fast - ema_slow
        signal = macd.ewm(span=9, adjust=False).mean()
        histogram = macd - signal
        return ema_fast, ema_slow, macd, signal, histogram

    def get_filtered_data(self):
        return self.filtered_data.to_dict()

    def get_macd_components(self):
        ema_fast, ema_slow, macd, signal, histogram = self.calculate_macd()
        return {
            'ema_fast': ema_fast.to_dict(),
            'ema_slow': ema_slow.to_dict(),
            'macd': macd.to_dict(),
            'signal': signal.to_dict(),
            'histogram': histogram.to_dict()
        }

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import mplfinance as mpf
from typing import Annotated

class MACDPlotter:
    def __init__(self, csv_file_path: str):
        self.data = pd.read_csv(csv_file_path, parse_dates=['date'])
        self.data.set_index('date', inplace=True)

    def filter_data(self, start_date: str, end_date: str):
        self.filtered_data = self.data.loc[start_date:end_date]

    def calculate_ema(self, period: int):
        return self.filtered_data['close'].ewm(span=period, adjust=False).mean()

    def calculate_macd(self):
        ema_fast = self.calculate_ema(12)
        ema_slow = self.calculate_ema(26)
        macd = ema_fast - ema_slow
        signal = macd.ewm(span=9, adjust=False).mean()
        histogram = macd - signal
        return ema_fast, ema_slow, macd, signal, histogram

    def plot_macd(self, save_path: str, plot_type: str = "candle", plot_style: str = "default", show_nontrading: bool = False):
        ema_fast, ema_slow, macd, signal, histogram = self.calculate_macd()

        ema_fast_plot = mpf.make_addplot(ema_fast, color='red', width=2.0, secondary_y=False)
        ema_slow_plot = mpf.make_addplot(ema_slow, color='green', width=2.0, secondary_y=False)

        histogram_positive = histogram.where(histogram >= 0)
        histogram_negative = histogram.where(histogram < 0)

        histogram_positive_plot = mpf.make_addplot(histogram_positive, panel=1, type='bar', color='green', secondary_y=False, width=0.8)
        histogram_negative_plot = mpf.make_addplot(histogram_negative, panel=1, type='bar', color='red', secondary_y=False, width=0.8)
        macd_plot = mpf.make_addplot(macd, panel=1, color='blue', width=2.0, secondary_y=True)
        signal_plot = mpf.make_addplot(signal, panel=1, color='magenta', width=2.0, secondary_y=True)
        apds = [ema_fast_plot, ema_slow_plot, histogram_positive_plot, histogram_negative_plot, macd_plot, signal_plot]

        plot_params = {
            "type": plot_type,
            "style": plot_style,
            "addplot": apds,
            "volume": True,
            "title": "MACD with EMA and Histogram",
            "ylabel": "Price",
            "ylabel_lower": "Volume",
            "show_nontrading": show_nontrading,
            "savefig": save_path,
            "figsize": (12, 8),
            "datetime_format": '%b %Y',
        }

        fig, axlist = mpf.plot(self.filtered_data, **plot_params, returnfig=True)

        fig.subplots_adjust(top=0.9)
        fig.suptitle('MACD with EMA and Histogram', y=0.95, fontsize=12)

        legend_lines = [
            axlist[0].plot([], [], color='red', linewidth=2, label='EMA Fast')[0],
            axlist[0].plot([], [], color='green', linewidth=2, label='EMA Slow')[0],
            axlist[2].plot([], [], color='blue', linewidth=2, label='MACD')[0],
            axlist[2].plot([], [], color='magenta', linewidth=2, label='Signal')[0],
        ]
        axlist[0].legend(handles=legend_lines[:2], loc='upper left')
        axlist[2].legend(handles=legend_lines[2:], loc='upper left')

        plt.close(fig)

        return f"MACD chart saved to {save_path}"


In [6]:
from typing import Optional

def plot_macd_tool(
    csv_file_path: Annotated[str, "Path to CSV file containing historical stock data"],
    start_date: Annotated[str, "Start date of the historical data in 'YYYY-MM-DD' format"],
    end_date: Annotated[str, "End date of the historical data in 'YYYY-MM-DD' format"],
    save_path: Annotated[str, "File path where the plot should be saved"],
    plot_type: Annotated[str, "Type of the plot (e.g., 'candle', 'line')"] = "candle",
    plot_style: Annotated[str, "Style of the plot (e.g., 'default', 'yahoo')"] = "default",
    show_nontrading: Annotated[bool, "Whether to show non-trading days on the chart"] = False
) -> str:
    plotter = MACDPlotter(csv_file_path)
    plotter.filter_data(start_date, end_date)
    return plotter.plot_macd(save_path, plot_type, plot_style, show_nontrading)

In [7]:
from IPython.display import Image, display
from typing import Annotated

def display_image_tool(file_path: Annotated[str, "Path to the image file to be displayed"]):
    """
    Displays an image from the given file path using IPython.display.
    """
    try:
        display(Image(filename=file_path))
        return f"Displayed image from {file_path}"
    except Exception as e:
        return f"Failed to display image from {file_path}: {e}"

In [8]:
import backtrader as bt
import pandas as pd
import json
import os
import matplotlib.pyplot as plt
from typing import Optional, Dict

class MACDStrategy(bt.Strategy):
    params = (('short_ema', 12), ('long_ema', 26), ('signal_ema', 9),)

    def __init__(self):
        self.macd = bt.indicators.MACD(
            self.data.close,
            period_me1=self.params.short_ema,
            period_me2=self.params.long_ema,
            period_signal=self.params.signal_ema
        )
        self.crossover = bt.indicators.CrossOver(self.macd.macd, self.macd.signal)

    def next(self):
        if self.crossover > 0:
            self.buy()
        elif self.crossover < 0:
            self.sell()

class BackTraderUtils:
    def back_test_macd(
        self,
        csv_file_path: Annotated[str, "Path to CSV file containing historical stock data"],
        start_date: Annotated[str, "Start date of the backtest in 'YYYY-MM-DD' format"],
        end_date: Annotated[str, "End date of the backtest in 'YYYY-MM-DD' format"],
        strategy_params: Annotated[str, "JSON string of strategy parameters, e.g., '{\"fast_ema\": 12, \"slow_ema\": 26, \"signal_ema\": 9}'"] = "",
        cash: Annotated[float, "Initial cash for backtesting"] = 10000.0,
        save_fig: Optional[Annotated[str, "File path to save the backtest result plot"]] = None,
    ) -> str:
        """
        Use the Backtrader library to backtest the MACD strategy on historical stock data from a CSV file.
        """
        cerebro = bt.Cerebro()
        strategy_class = MACDStrategy
        try:
            strategy_params_dict = json.loads(strategy_params) if strategy_params else {}
        except json.JSONDecodeError:
            return "Error: Invalid JSON format for strategy_params."

        cerebro.addstrategy(strategy_class, **strategy_params_dict)

        # Read data from CSV file
        try:
            df = pd.read_csv(csv_file_path, parse_dates=['date'])
        except FileNotFoundError:
            return f"Error: File not found at {csv_file_path}."
        except pd.errors.ParserError:
            return "Error: Failed to parse CSV file."

        df['datetime'] = pd.to_datetime(df['date'])
        df.set_index('datetime', inplace=True)
        df = df[['open', 'high', 'low', 'close', 'volume']]

        # Filter the data based on the start and end date
        df = df.loc[start_date:end_date]
        
        # Ensure there are enough data points for the MACD calculation
        if len(df) < max(strategy_params_dict.get('short_ema', 12), strategy_params_dict.get('long_ema', 26), strategy_params_dict.get('signal_ema', 9)):
            return "Error: Not enough data available for the specified date range to calculate MACD."

        if df.empty:
            return "Error: No data available for the specified date range."
        
        # Rename columns to match Backtrader expectations
        df.columns = ['Open', 'High', 'Low', 'Close', 'Volume']

        # Create a data feed
        data = bt.feeds.PandasData(dataname=df)
        cerebro.adddata(data)  # Add the data feed
        
        # Set our desired cash start
        cerebro.broker.setcash(cash)

        # Attach analyzers
        cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name="sharpe_ratio")
        cerebro.addanalyzer(bt.analyzers.DrawDown, _name="draw_down")
        cerebro.addanalyzer(bt.analyzers.Returns, _name="returns")
        cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="trade_analyzer")

        stats_dict: Dict[str, any] = {"Starting Portfolio Value": cerebro.broker.getvalue()}

        results = cerebro.run()  
        first_strategy = results[0]

        # Access analysis results
        stats_dict["Final Portfolio Value"] = cerebro.broker.getvalue()
        stats_dict["Sharpe Ratio"] = (
            first_strategy.analyzers.sharpe_ratio.get_analysis()
        )
        stats_dict["Drawdown"] = first_strategy.analyzers.draw_down.get_analysis()
        stats_dict["Returns"] = first_strategy.analyzers.returns.get_analysis()
        stats_dict["Trade Analysis"] = (
            first_strategy.analyzers.trade_analyzer.get_analysis()
        )

        if save_fig:
            directory = os.path.dirname(save_fig)
            if directory:
                os.makedirs(directory, exist_ok=True)
            plt.figure(figsize=(16, 10))
            cerebro.plot()
            plt.savefig(save_fig)
            plt.close()

        return "Back Test Finished. Results: \n" + json.dumps(stats_dict, indent=2)

In [9]:
def backtest_macd_tool(
    csv_file_path: Annotated[str, "Path to CSV file containing historical stock data"],
    start_date: Annotated[str, "Start date of the backtest in 'YYYY-MM-DD' format"],
    end_date: Annotated[str, "End date of the backtest in 'YYYY-MM-DD' format"],
    strategy_params: Annotated[str, "JSON string of strategy parameters, e.g., '{\"fast_ema\": 12, \"slow_ema\": 26, \"signal_ema\": 9}'"] = "",
    cash: Annotated[float, "Initial cash for backtesting"] = 10000.0,
    save_fig: Optional[Annotated[str, "File path to save the backtest result plot"]] = None,
) -> str:
    """
    Backtests a MACD strategy on historical stock data from a CSV file.
    """
    utils = BackTraderUtils()
    return utils.back_test_macd(csv_file_path, start_date, end_date, strategy_params, cash, save_fig)

In [10]:
strategist = MultimodalConversableAgent(
    name="Trade_Strategist",
    system_message=dedent(
        """
        You are a trading strategist who inspects financial charts and optimizes trading strategies.
        You have been tasked with developing a Moving average convergence/divergence (MACD) trading strategy.
        You have the following main actions to take:
        1. Ask the backtesting analyst to plot historical stock price data with designated MACD indicators.
        2. Inspect the stock price chart carefully and determine MACD (short_ema,long_ema and signal_ema) parameters.
        3. Highlight the points/periods where the MACD line crosses the Signal line and interpret their significance critically.
        4. Ask the backtesting analyst to backtest the MACD trading strategy with designated parameters to evaluate its performance.
        5. Inspect the backtest result and optimize the MACD parameters based on the returned results.
        6. Avoid testing the same parameters more than 1 time to ensure efficiency.
        Reply TERMINATE when you think the strategy is good enough.
        """
    ),
    llm_config=llm_config_4v,
)

analyst = AssistantAgent(
    name="Backtesting_Analyst",
    system_message=dedent(
        f"""
        You are a backtesting analyst with a strong command of quantitative analysis tools.
        You have two main tasks to perform, choose one each time you are asked by the trading strategist:
        1. Plot historical stock price data for {company} in the file at {file_path} with MACD indicators (short_ema,long_ema and signal_ema) according to the trading strategist's need.
        2. Backtest the MACD trading strategy with designated parameters (short_ema,long_ema and signal_ema) and save the results as an image file.
        For both tasks, after the tool calling, you should do as follows:
            1. display the created & saved image file using the `display_image_tool` tool;
            2. Assume the saved image file is "test.png", reply as follows: "Optimize the MACD parameters(short_ema,long_ema and signal_ema) based on this image <img test.png>. TERMINATE".
        """
    ),
    llm_config=llm_config,
)

analyst_executor = UserProxyAgent(
    name="Backtesting_Analyst_Executor",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").find("TERMINATE") >= 0,
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": "coding",
        "use_docker": False,
    },
)

In [11]:
from autogen import register_function

# Register the plot_macd_tool function
register_function(
    plot_macd_tool,
    caller=analyst,  # The analyst agent can suggest calls to the plot_macd_tool function.
    executor=analyst_executor,  # The analyst_executor agent can execute the plot_macd_tool calls.
    name="plot_macd_tool",  # By default, the function name is used as the tool name.
    description="Plots the MACD chart with EMA, MACD histogram, and signals, and saves the plot to a file.",  # Description of the tool.
)

# Register the backbacktest_macd_tool_test_macd function
register_function(
    backtest_macd_tool,
    caller=analyst,  # The analyst agent can suggest calls to the backtest_macd_tool function.
    executor=analyst_executor,  # The analyst_executor agent can execute the backtest_macd_tool calls.
    name="backtest_macd_tool",  # By default, the function name is used as the tool name.
    description="Backtests a MACD trading strategy on historical stock data from a CSV file.",  # Description of the tool.
)

from autogen import register_function

# Register the display_image_tool function
register_function(
    display_image_tool,
    caller=analyst,  # Analyst agent can suggest calls to the display_image_tool function.
    executor=analyst_executor,  # The analyst_executor agent can execute the display_image_tool calls.
    name="display_image_tool",  # By default, the function name is used as the tool name.
    description="Displays an image from the given file path using IPython.display.",  # Description of the tool.
)

The return type of the function 'display_image_tool' is not annotated. Although annotating it is optional, the function should return either a string, a subclass of 'pydantic.BaseModel'.


In [12]:
def reflection_message_analyst(recipient, messages, sender, config):
    print("Reflecting strategist's response ...")
    last_msg = recipient.chat_messages_for_summary(sender)[-1]["content"]
    return (
        "Message from Trade Strategist is as follows:"
        + last_msg
        + "\n\nBased on this information, conduct a backtest on the specified stock and MACD strategy, and report your backtesting results back to the strategist."
    )

user_proxy = UserProxyAgent(
    name="User_Proxy",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").endswith("TERMINATE"),
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": "coding",
        "use_docker": False,
    },
)

user_proxy.register_nested_chats(
    [
        {
            "sender": analyst_executor,
            "recipient": analyst,
            "message": reflection_message_analyst,
            "max_turns": 10,
            "summary_method": "last_msg",
        }
    ],
    trigger=strategist,
)

In [13]:
task = dedent(f"""
    Based on {company}'s stock data from {start_date} to {end_date} in the file at {file_path}, determine the possible optimal parameters for a MACD Strategy over this period.
    First, ask the analyst to plot a candlestick chart of the stock price data with MACD indicators to visually carefully inspect the price movements and make an initial assessment.
    Then, ask the analyst to backtest the MACD (short_ema,long_ema and signal_ema) strategy parameters using the backtesting tool, and report results back for further optimization.
""")

# Start logging with logger_type and the filename to log to
# logging_session_id = autogen.runtime_logging.start(logger_type="file", config={"filename": "runtime.log"})
# print("Logging session ID: " + str(logging_session_id))

with Cache.disk() as cache:
    user_proxy.initiate_chat(
        recipient=strategist, message=task, max_turns=5, summary_method="last_msg"
    )

# autogen.runtime_logging.stop()

User_Proxy (to Trade_Strategist):


Based on AAKASH's stock data from 2022-01-01 to 2024-01-01 in the file at ../../data/DailyData/AAKASH.csv, determine the possible optimal parameters for a MACD Strategy over this period.
First, ask the analyst to plot a candlestick chart of the stock price data with MACD indicators to visually carefully inspect the price movements and make an initial assessment.
Then, ask the analyst to backtest the MACD (short_ema,long_ema and signal_ema) strategy parameters using the backtesting tool, and report results back for further optimization.


--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Trade_Strategist (to User_Proxy):

Sure, let's proceed step by step.

### Step 1: Request the Analyst to Plot the Chart

**To the Backtesting Analyst:**

Please plot a candlestick chart of AAKASH's stock price data from 2022-01-01 to 2024-01-01 using the file located at `../../data/DailyData/AAKASH.csv`. Add

[runtime logging] log_function_use: autogen logger is None


Backtesting_Analyst_Executor (to Backtesting_Analyst):

Backtesting_Analyst_Executor (to Backtesting_Analyst):

***** Response from calling tool (call_kmrr6VBGZEJBF4eeCnzaeWW7) *****
MACD chart saved to aakash_macd_chart.png
**********************************************************************

--------------------------------------------------------------------------------


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}